<a href="https://colab.research.google.com/github/aetev/Hearth-Stone-Python-Simulator/blob/main/card_enc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [142]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [143]:
import json
import tensorflow as tf
import re
from tensorflow.keras.layers import Embedding, TextVectorization
# Assuming AtomicCards.json is in your Google Drive, adjust the path if necessary
file_path = '/content/drive/MyDrive/MTGdata/AtomicCards.json'


with open(file_path, 'r') as f:
  data = json.load(f)['data']

In [144]:
def filter_legal_cards(data):
    legal_commander_data = {}
    for card_name, card_data in data.items():
        try:
            if card_data[0]['legalities']['commander'] == 'Legal':
                legal_commander_data[card_name] = card_data[0] # Store only the first element of card_data
        except (KeyError, IndexError):
            pass  # Or print a message, log it, etc.
    return legal_commander_data

filtered_data = filter_legal_cards(data)


In [186]:
items = list(filtered_data.items())

text_list = []
for row in items[:40]:
  second_key, second_value = row
  if second_value['text'] != None:
    text_list.append(second_value['text'])


for text in text_list:
  print(text)

This creature enters tapped.
When this creature dies, open an Attraction.
Whenever you roll to visit your Attractions, if you roll a 6, you may return this card from your graveyard to the battlefield.
When this creature enters from anywhere other than a graveyard or exile, if it's on the battlefield and you control 9 or fewer creatures named "Name Sticker" Goblin, roll a 20-sided die.
1-6 | Add {R}{R}{R}{R}.
7-14 | Add {R}{R}{R}{R}{R}.
15-20 | Add {R}{R}{R}{R}{R}{R}.
Equipped creature gets +2/+2.
Equip {3} ({3}: Attach to target creature you control. Equip only as a sorcery.)
Tap up to two target creatures.
Whenever you put a name sticker on a creature, you may return this card from your graveyard to your hand.
When this enchantment enters, create a 1/1 white Human creature token, a 1/1 blue Merfolk creature token, and a 1/1 red Goblin creature token. Then secretly choose Human, Merfolk, or Goblin.
Sacrifice this enchantment, Reveal the creature type you chose: If target attacking crea

In [199]:
class TextCleaningLayer(tf.keras.layers.Layer):
    def __init__(self,sentence_list, **kwargs):
        super(TextCleaningLayer, self).__init__(**kwargs)
        self.vectorizer = TextVectorization(standardize=None, ngrams=2, output_mode='int',output_sequence_length=150)
        self.vectorizer.adapt(self.clean_text(sentence_list))
        self.embedding_layer = Embedding(input_dim=3000, output_dim=100)

    def clean_text(self, text):

        #text = card['text']
        #card_name = card['name']


        # Convert input to string tensor if it's not already
        if not isinstance(text, tf.Tensor):
            text = tf.convert_to_tensor(text, dtype=tf.string)

        cleaned_text = tf.strings.lower(text)
        #cleaned_text = tf.strings.regex_replace(cleaned_text, card_name, 'self')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\n', ' ')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '[\/\.]', ' ')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '[\"—"]', ' ')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\{|\}', ' ')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\([^)]*\)', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '[^\w\s+/\-]', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\s+', ' ')
        return cleaned_text

    def call(self, inputs):
        cleaned_text = self.clean_text(inputs)
        vectorized_text = self.vectorizer(cleaned_text)
        embedded_text = self.embedding_layer(vectorized_text)
        return cleaned_text


# Example usage with a batch of texts
batch_texts = tf.constant(text_list)
cleaned_text_layer = TextCleaningLayer(batch_texts)
cleaned_batch = cleaned_text_layer.call(batch_texts)
for i in range(len(cleaned_batch)):
  tf.print(cleaned_batch[i])




this creature enters tapped when this creature dies open an attraction whenever you roll to visit your attractions if you roll a 6 you may return this card from your graveyard to the battlefield 
when this creature enters from anywhere other than a graveyard or exile if its on the battlefield and you control 9 or fewer creatures named name sticker goblin roll a 20-sided die 1-6 add r r r r 7-14 add r r r r r 15-20 add r r r r r r 
equipped creature gets +2 +2 equip 3 
tap up to two target creatures whenever you put a name sticker on a creature you may return this card from your graveyard to your hand 
when this enchantment enters create a 1 1 white human creature token a 1 1 blue merfolk creature token and a 1 1 red goblin creature token then secretly choose human merfolk or goblin sacrifice this enchantment reveal the creature type you chose if target attacking creature token is the chosen type put three +1 +1 counters on it and it gains deathtouch until end of turn 
casualty 1 look a

In [185]:
print(cleaned_text_layer.vectorizer.get_vocabulary())

['', '[UNK]', 'r', 'creature', 'r r', 'your', 'you', '1', 'a', 'this', 'the', 'of', 'when', 'token', 'or', 'on', 'enters', 'creature token', 'and', 'when this', 'to', 'this creature', 'put', 'graveyard', 'from', 'a 1', '1 1', '+2', '+1', 'your hand', 'your graveyard', 'you control', 'whenever', 'turn', 'roll', 'if', 'hand', 'goblin', 'from your', 'creatures', 'creature enters', 'control', 'card', 'add r', 'add', 'your library', 'you roll', 'you may', 'x', 'whenever you', 'type', 'two', 'this enchantment', 'this card', 'the battlefield', 'target', 'sticker', 'roll a', 'return this', 'return', 'put a', 'other', 'on the', 'of your', 'name sticker', 'name', 'merfolk', 'may return', 'may', 'life', 'library', 'it', 'is the', 'is', 'insect', 'human', 'have', 'graveyard to', 'enters create', 'enchantment', 'each', 'dies', 'create a', 'create', 'card from', 'battlefield', 'aatchik', '6', '20', '+2 +2', '+1 +1', 'your turn', 'your opponents', 'your attractions', 'you take', 'you put', 'you chose